In [1]:
# %matplotlib widget
# %load_ext autoreload
# %autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.util.dbx import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from decouple import Config, RepositoryEnv

DOTENV_FILE = os.path.dirname(os.path.dirname(os.path.abspath(''))) + "/local.env"
env_config = Config(RepositoryEnv(DOTENV_FILE))

/home/ipausers/bisot/pycode/MscThesis/amftrack/util/dbx.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
directory = r'/mnt/sun-temp/TEMP/PRINCE_analysis/'

update_plate_info(directory,local=True)
all_folders = get_current_folders(directory,local=True)

analysed:   0%|          | 0/137 [00:00<?, ?it/s]

In [3]:
folders = all_folders.loc[all_folders['folder']=='20220510_0339_Plate03']

In [4]:
dir_drop = "test"

In [13]:
def upload_zip(path_total,target,catch_exception=True):
    '''
    Handles upload of files and folders taking only a path as an argument. If the total path is a
    file it is uploaded directly. Otherwise the folder is zipped and uploaded. Upload of individual files is
    inneficient.
     '''
    if os.path.isdir(path_total):
        stamp = time_ns()
        path_zip = f'{temp_path}/{stamp}.zip'
        zip_file(path_total, path_zip)
        upload(path_zip, target,
               chunk_size=256 * 1024 * 1024,catch_exception=catch_exception)
        os.remove(path_zip)

    else:
        upload(path_total, target,
               chunk_size=256 * 1024 * 1024,catch_exception=catch_exception)

def sync_fold(origin,target):
    '''
    Launches a sync between two directory. Will only work on linux system.
     '''
    cmd = f'rsync --update -avh {origin} {target}'
    # print(cmd)
    call(cmd,shell=True)

def upload_folders(folders: pd.DataFrame,dir_drop = 'DATA',catch_exception=True):
    '''
    Upload all the folders in the dataframe to a location on dropbox
     '''
    run_info = folders.copy()
    folder_list = list(run_info['folder'])
    with tqdm(total=len(folder_list), desc="transferred") as pbar:
        for folder in folder_list:
            directory_name = folder
            run_info['unique_id'] = run_info['Plate'].astype(str) + "_" + run_info['CrossDate'].astype(str)
            line = run_info.loc[run_info['folder'] == directory_name]
            id_unique = line['unique_id'].iloc[0]

            path_snap = line['total_path'].iloc[0]
            path_info = f'{temp_path}/{directory_name}_info.json'
            for subfolder in os.listdir(path_snap):
                line.to_json(path_info)
                path_total = os.path.join(path_snap, subfolder)
                suffix = '.zip' if os.path.isdir(path_total) else ''
                target = f'/{dir_drop}/{id_unique}/{directory_name}/{subfolder}{suffix}'
                upload_zip(path_total,target,catch_exception=catch_exception)
            upload(path_info, f'/{dir_drop}/{id_unique}/{directory_name}_info.json', chunk_size=256 * 1024 * 1024,catch_exception=catch_exception)
            os.remove(path_info)
            pbar.update(1)
            
upload_folders(folders,'test',catch_exception=False)

transferred:   0%|          | 0/1 [00:00<?, ?it/s]

2022-05-13 12:20:14,245-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 12:20:14,447-[INFO]- dropbox:477 -> Request to files/upload
2022-05-13 12:20:15,807-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 12:20:16,016-[INFO]- dropbox:477 -> Request to files/upload


Uploaded:   0%|          | 0/1207394518 [00:00<?, ?it/s]

2022-05-13 12:20:19,747-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 12:20:19,940-[INFO]- dropbox:477 -> Request to files/upload_session/start
2022-05-13 12:20:29,632-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 12:20:38,003-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 12:20:47,449-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 12:20:56,027-[INFO]- dropbox:477 -> Request to files/upload_session/finish


zipping:   0%|          | 0/150 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/881136983 [00:00<?, ?it/s]

2022-05-13 12:22:11,096-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 12:22:11,298-[INFO]- dropbox:477 -> Request to files/upload_session/start
2022-05-13 12:22:22,709-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 12:22:35,026-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 12:22:44,875-[INFO]- dropbox:477 -> Request to files/upload_session/finish


zipping:   0%|          | 0/152 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/747685765 [00:00<?, ?it/s]

2022-05-13 12:24:18,593-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 12:24:18,793-[INFO]- dropbox:477 -> Request to files/upload_session/start
2022-05-13 12:24:26,837-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 12:24:34,419-[INFO]- dropbox:477 -> Request to files/upload_session/finish
2022-05-13 12:24:42,590-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 12:24:42,786-[INFO]- dropbox:477 -> Request to files/upload


In [5]:
def upload_zip(path_total,target,trhesh = 4 * 1024 * 1024):
    if os.path.isdir(path_total):
        stamp = time_ns()
        path_zip = f'{temp_path}/{stamp}.zip'
        zip_file(path_total, path_zip)
        upload(path_zip, target,
               chunk_size=256 * 1024 * 1024)
        os.remove(path_zip)

    else:
        upload(path_total, target,
               chunk_size=256 * 1024 * 1024)

In [7]:
run_info = folders.copy()
folder_list = list(run_info['folder'])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info['unique_id'] = run_info['Plate'].astype(str) + "_" + run_info['CrossDate'].astype(str)
        line = run_info.loc[run_info['folder'] == directory_name]
        id_unique = line['unique_id'].iloc[0]

        path_snap = line['total_path'].iloc[0]
        path_info = f'{temp_path}/{directory_name}_info.json'

        for subfolder in os.listdir(path_snap):
            path_total = os.path.join(path_snap, subfolder)
            size = os.path.getsize(path_total)
            path_zip = f'{temp_path}/{directory_name}_{subfolder}.zip'
            line.to_json(path_info)
            zip_file(path_total, path_zip)
            upload(path_zip, f'/{dir_drop}/{id_unique}/{directory_name}/{subfolder}.zip',
                   chunk_size=256 * 1024 * 1024)
        upload(path_info, f'/{dir_drop}/{id_unique}/{directory_name}_info.json', chunk_size=256 * 1024 * 1024)
        os.remove(path_info)
        os.remove(path_zip)
        pbar.update(1)

transferred:   0%|          | 0/1 [00:00<?, ?it/s]

zipping: 0it [00:00, ?it/s]

2022-05-13 11:40:20,421-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 11:40:20,630-[INFO]- dropbox:477 -> Request to files/upload


zipping: 0it [00:00, ?it/s]

2022-05-13 11:40:22,070-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 11:40:22,282-[INFO]- dropbox:477 -> Request to files/upload


...

zipping:   0%|          | 0/150 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/881136983 [00:00<?, ?it/s]

2022-05-13 11:41:31,436-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 11:41:31,648-[INFO]- dropbox:477 -> Request to files/upload_session/start
2022-05-13 11:41:38,658-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 11:41:47,148-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 11:41:53,708-[INFO]- dropbox:477 -> Request to files/upload_session/finish


zipping:   0%|          | 0/152 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
DOTENV_FILE

In [ ]:
directory = '/mnt/sun-temp/TEMP/PRINCE_syncing/'
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']=='860']

In [ ]:
p = dropbox.common.PathRoot.namespace_id("<TEAM_FOLDER_ID>")
dbx = dropbox.Dropbox(API, timeout=900).with_path_root(p)

In [ ]:
dbx = dropbox.DropboxTeam(app_key = "///",
            app_secret = "///", oauth2_refresh_token = '///-//')
for entry in dbx.team_team_folder_list().team_folders:
    print(entry)
p = dropbox.common.PathRoot.namespace_id("///")
dbx = dbx.with_path_root(p)
dbx = dbx.as_user("///-///")

In [ ]:
dbx.team_members_list_v2()

In [ ]:
dbx.files_upload(open("automated_stitch.py", "rb").read(), "/DATA/test", mode=dropbox.files.WriteMode.overwrite)

In [ ]:
for entry in dbx.team_namespaces_list().namespaces:
    print(entry.namespace_id +" / "+entry.name)

In [ ]:
p = dropbox.common.PathRoot.namespace_id("")
dbx = dbx.with_path_root(p)

In [ ]:
for entry in dbx.team_namespaces_list().namespaces:
    print(entry.namespace_id +" / "+entry.name)

In [76]:
import dropbox
from dropbox import DropboxOAuth2FlowNoRedirect

'''
Populate your app key in order to run this locally
'''
APP_KEY = "gm3xirr1cgdjfhi"

auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY, use_pkce=True, token_access_type='offline')

authorize_url = auth_flow.start()
print("1. Go to: " + authorize_url)
print("2. Click \"Allow\" (you might have to log in first).")
print("3. Copy the authorization code.")
auth_code = input("Enter the authorization code here: ").strip()

try:
    oauth_result = auth_flow.finish(auth_code)
except Exception as e:
    print('Error: %s' % (e,))
    exit(1)

with dropbox.Dropbox(oauth2_refresh_token=oauth_result.refresh_token, app_key=APP_KEY) as dbx:
    print(dbx)
    # # p = dropbox.common.PathRoot.namespace_id("<TEAM_FOLDER_ID>")
    # dbx = dbx.with_path_root(p)    
    print(dbx,"Successfully set up client!")

1. Go to: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=gm3xirr1cgdjfhi&token_access_type=offline&code_challenge=1F8bJvz6vGuWhdKnTF4Jq4PsuJbQBJo0w9QdkBNDMDs&code_challenge_method=S256
2. Click "Allow" (you might have to log in first).
3. Copy the authorization code.


Enter the authorization code here:  x9vxVXoO1WcAAAAAAAAB-Wqfee0rzWhTmzEvIfh0ZTg


<dropbox.dropbox_client.Dropbox object at 0x7f1a470d1a00> Successfully set up client!


In [79]:
oauth_result.refresh_token

'JTGUKclu02UAAAAAAAAAATreksjJSYkv2dIaRosB73NCVqZMgRvnDAaBWQAjV5w-'

In [ ]:
with dropbox.Dropbox(oauth2_refresh_token=oauth_result.refresh_token, app_key=APP_KEY) as dbx:
    # # p = dropbox.common.PathRoot.namespace_id("<TEAM_FOLDER_ID>")
    # dbx = dbx.with_path_root(p)    
    print(dbx,"Successfully set up client!")

In [ ]:
env_config = Config(RepositoryEnv(DOTENV_FILE))
dir_drop = '/test'
# API = env_config.get("API_KEY")
API = ""
temp_path = env_config.get("TEMP_PATH")


In [ ]:
with open("sync_prince.py", "rb") as f:
    dbx.files_upload(f.read(), '/test/test.py', mode=dropbox.files.WriteMode.overwrite)

In [ ]:
upload(API,"sync_prince.py",'/test/test.py')

In [ ]:
os.path.join(path_snap,subfolder)